<a href="https://colab.research.google.com/github/M-Abbi/Financial-Modeling/blob/main/Option_Pricing_via_Cox_Ross_Rubinstein_(CRR)_Binomial_Tree_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Binomial Model Fundamentals:**

**Discrete Time Steps:** The model divides the time to expiration into discrete
steps.

**Up/Down Movements:** At each step, the underlying asset price can move up (u) or down (d).

**Risk-Neutral Probability** (q): The probability of an up-move in a risk-neutral world.

**No Arbitrage**: d < (1 + r_dt) < u, where r_dt is the risk-free rate per step.

**Payoff at Expiration**: The option's value at the final nodes is known (e.g., max(S_T - K, 0) for a call).

**Backward Induction**: Work backward from expiration to the present to find the option's current price. At each node, the option value is the discounted expected value of the option from the next step, under risk-neutral probabilities.

**Early Exercise** (for American options): At each node before expiration, compare the discounted expected future value with the intrinsic value if exercised immediately. The option value is the maximum of these two.


**Key Parameters**: Please identify the inputs needed:


*   S0: Current stock price
*   K: Strike price
*   T: Time to expiration (in years)
*   r: Risk-free interest rate (annualized)
*   sigma: Volatility (annualized standard deviation of log returns)
*   N: Number of time steps in the tree
*   option_type: 'call' or 'put'
*   exercise_type: 'european' or 'american' (important for backward induction)

In [2]:
import numpy as np

def binomial_option_pricer(S0, K, T, r, sigma, N, option_type='call', exercise_type='european'):
    """
    Prices an option using the Cox-Ross-Rubinstein (CRR) Binomial Tree Model.

    Parameters:
    S0 (float): Current stock price
    K (float): Strike price
    T (float): Time to expiration (in years)
    r (float): Risk-free interest rate (annualized, e.g., 0.05 for 5%)
    sigma (float): Volatility of the underlying stock (annualized, e.g., 0.2 for 20%)
    N (int): Number of time steps in the binomial tree
    option_type (str): 'call' or 'put'
    exercise_type (str): 'european' or 'american'

    Returns:
    float: Estimated option price
    """

    # --- 1. Calculate intermediate parameters ---
    dt = T / N  # Time step duration
    u = np.exp(sigma * np.sqrt(dt))  # Up-factor
    d = 1 / u  # Down-factor (ensures recombinant tree)
    # d = np.exp(-sigma * np.sqrt(dt)) # Alternative for d

    # Risk-neutral probability of an up-move
    q = (np.exp(r * dt) - d) / (u - d)
    # Discount factor for one time step
    discount_factor = np.exp(-r * dt)

    # --- 2. Initialize asset prices at each node (forward pass) ---
    # Stock price tree: stock_prices[i, j] is price at time step i, after j up-moves
    stock_prices = np.zeros((N + 1, N + 1))
    for i in range(N + 1):  # Time steps from 0 to N
        for j in range(i + 1):  # Number of up-moves (0 to i)
            stock_prices[i, j] = S0 * (u**j) * (d**(i - j))

    # --- 3. Initialize option values at expiration (last time step) ---
    # Option value tree: option_values[i, j] is option value at time step i, after j up-moves
    option_values = np.zeros((N + 1, N + 1))

    if option_type == 'call':
        option_values[N, :] = np.maximum(0, stock_prices[N, :] - K)
    elif option_type == 'put':
        option_values[N, :] = np.maximum(0, K - stock_prices[N, :])
    else:
        raise ValueError("option_type must be 'call' or 'put'")

    # --- 4. Backward induction to calculate option values at earlier nodes ---
    for i in range(N - 1, -1, -1):  # Iterate backwards from N-1 to 0
        for j in range(i + 1):  # Number of up-moves (0 to i)
            # Expected option value in the next period
            expected_value_future = q * option_values[i + 1, j + 1] + \
                                    (1 - q) * option_values[i + 1, j]

            # Discounted expected value (value if held)
            value_if_held = discount_factor * expected_value_future

            if exercise_type == 'european':
                option_values[i, j] = value_if_held
            elif exercise_type == 'american':
                # Intrinsic value if exercised at current node (i, j)
                if option_type == 'call':
                    intrinsic_value_now = np.maximum(0, stock_prices[i, j] - K)
                else: # put
                    intrinsic_value_now = np.maximum(0, K - stock_prices[i, j])
                option_values[i, j] = np.maximum(value_if_held, intrinsic_value_now)
            else:
                raise ValueError("exercise_type must be 'european' or 'american'")

    return option_values[0, 0]



    european_put_itmp = binomial_option_pricer(S0_itmp, K_itmp, T_itmp, r_itmp, sigma_itmp, N_steps_itmp,
                                               option_type='put', exercise_type='european')
    print(f"European Put (ITM) Option Price (for comparison): {european_put_itmp:.4f}")
    # For puts, American can be > European due to early exercise premium

In [3]:
# --- Example Usage ---
if __name__ == "__main__":
    # Market and option parameters
    S0_val = 100      # Current stock price
    K_val = 100       # Strike price
    T_val = 1         # Time to expiration (1 year)
    r_val = 0.05      # Risk-free rate (5%)
    sigma_val = 0.2   # Volatility (20%)
    N_steps = 100     # Number of steps in the binomial tree (more steps = more accuracy)

    # Price a European Call
    european_call_price = binomial_option_pricer(S0_val, K_val, T_val, r_val, sigma_val, N_steps,
                                                 option_type='call', exercise_type='european')
    print(f"European Call Option Price: {european_call_price:.4f}")

    # Price a European Put
    european_put_price = binomial_option_pricer(S0_val, K_val, T_val, r_val, sigma_val, N_steps,
                                                option_type='put', exercise_type='european')
    print(f"European Put Option Price: {european_put_price:.4f}")

    # Price an American Call
    # Note: For non-dividend paying stocks, American call price = European call price
    american_call_price = binomial_option_pricer(S0_val, K_val, T_val, r_val, sigma_val, N_steps,
                                                 option_type='call', exercise_type='american')
    print(f"American Call Option Price: {american_call_price:.4f}")

    # Price an American Put
    american_put_price = binomial_option_pricer(S0_val, K_val, T_val, r_val, sigma_val, N_steps,
                                                option_type='put', exercise_type='american')
    print(f"American Put Option Price: {american_put_price:.4f}")

    # --- Example with different parameters (e.g., In-the-money Put) ---
    print("\n--- In-the-money American Put Example ---")
    S0_itmp = 90       # Current stock price (lower than strike)
    K_itmp = 100       # Strike price
    T_itmp = 0.5       # Time to expiration (6 months)
    r_itmp = 0.02      # Risk-free rate (2%)
    sigma_itmp = 0.3   # Volatility (30%)
    N_steps_itmp = 150

    american_put_itmp = binomial_option_pricer(S0_itmp, K_itmp, T_itmp, r_itmp, sigma_itmp, N_steps_itmp,
                                               option_type='put', exercise_type='american')
    print(f"American Put (ITM) Option Price: {american_put_itmp:.4f}")

European Call Option Price: 10.4306
European Put Option Price: 5.5536
American Call Option Price: 10.4306
American Put Option Price: 6.0824

--- In-the-money American Put Example ---
American Put (ITM) Option Price: 13.4292
